In [ ]:
using System;
using System.Collections.Generic;
using System.IO;
using System.Text.Json;
using Sinopac.Shioaji;


class SJCls
{
        private static Shioaji _api = new Shioaji();
        public void Login()
        {
            string jsonString = File.ReadAllText("Sinopac.json");
            JsonElement root = JsonDocument.Parse(jsonString).RootElement;
            _api.Login(root.GetProperty("API_Key").GetString(), root.GetProperty("Secret_Key").GetString());
            _api.ca_activate("Sinopac.pfx", root.GetProperty("ca_passwd").GetString(), root.GetProperty("person_id").GetString());
        }

        public dynamic TbarTseTxfOtc()
        {
            SJList res = _api.Snapshots(new List<IContract>() {
            _api.Contracts.Futures["TXF"]["TXFR1"],
            _api.Contracts.Indexs["TSE"]["001"],
            _api.Contracts.Indexs["OTC"]["101"]
            });

            List<Dictionary<string, List<object>>> combinedList = new List<Dictionary<string, List<object>>>();
            for (int i = 0; i < res.ToArray().Length; i++)
            {
                Dictionary<string, List<object>> dict = new Dictionary<string, List<object>>();
                List<object> values = new List<object>();
                values.Add(res[0].close);
                values.Add(res[0].change_price);
                values.Add(res[0].change_rate);
                dict.Add(res[i].code, values);
                combinedList.Add(dict);
            }
            return combinedList;
        }

        public SJList AmountRank(int num)
        {
            SJList res = _api.Scanners(date: "2023-12-29", scannerType: ScannerType.AmountRank, count: num);
            return res;
        }

        public SJList ChangePercentRank(int num)
        {
            SJList res = _api.Scanners(date: "2023-12-29", scannerType: ScannerType.ChangePercentRank, count: num);
            return res;
        }

        public object TicksLastCount(int num)
        {
            var TicksQuery = _api.Ticks(_api.Contracts.Futures["TXF"]["TXFR1"], "2023-12-29", TicksQueryType.LastCount, last_cnt: num);

            List<DateTimeOffset> generalTimes = new List<DateTimeOffset>();
            foreach (long timestamp in TicksQuery.ts)
            {
                DateTime dateTime = DateTimeOffset.FromUnixTimeMilliseconds(timestamp / 1000000).UtcDateTime;
                generalTimes.Add(new DateTimeOffset(dateTime, TimeSpan.Zero));
            }
            List<string> tsFormat = new List<string>();
            foreach (var i in generalTimes) tsFormat.Add(i.ToString("HH:mm:ss"));

            List<Dictionary<string, object>> combinedList = new List<Dictionary<string, object>>();
            for (int i = 0; i < tsFormat.ToArray().Length; i++)
            {
                Dictionary<string, object> dict = new Dictionary<string, object>();
                dict.Add(tsFormat[i], TicksQuery.close[i]);
                combinedList.Add(dict);
            }
            return combinedList;
        }

        public List<object> OpPremium(string OptionWeek, string YearMonth, string upper, string lower)
        {
            // 到時用成交價+50再除100無條件捨報後，再乘100  相對地減100
            var res = _api.Snapshots(new List<IContract>() {
                _api.Contracts.Options[OptionWeek][OptionWeek + YearMonth + lower + "C"],
                _api.Contracts.Options[OptionWeek][OptionWeek + YearMonth + upper + "C"],
                _api.Contracts.Options[OptionWeek][OptionWeek + YearMonth + upper + "P"],
                _api.Contracts.Options[OptionWeek][OptionWeek + YearMonth + lower + "P"],
                });

            List<object> op = new List<object>();
            op.Add(res[0].code);
            op.Add(res[0].sell_price);
            op.Add(res[1].code);
            op.Add(res[1].buy_price);
            op.Add(res[2].code);
            op.Add(res[2].sell_price);
            op.Add(res[3].code);
            op.Add(res[3].buy_price);

            return new List<object> {op[1], op[3], op[5], op[7], 
                $"BCSC:{(double)op[1]-(double)op[3]}、BPSP: {(double)op[5]-(double)op[7]}" };
        }


        public SJList BlueChips()
        {
            List<string> tw50 = new List<string>(){
                "2330", "2454", "2317", "2412", "2382", "2881", "2308", "6505", "2882", "2303",
                "3711", "2886", "2891", "1303", "1301", "2002", "1216", "2884", "2207", "5880",
                "3008", "2892", "3045", "1326", "2357", "2885", "6669", "2395", "3034", "5871",
                "2880", "2603", "2345", "3231", "4904", "2912", "2301", "3037", "1101", "2327",
                "2890", "2379", "2408", "3661", "3443", "4938", "5876", "2887", "2883", "1590"
            };
            var tw50_otc10 = new List<IContract>();
            foreach (var i in tw50)
            {
                tw50_otc10.Add(_api.Contracts.Stocks["TSE"][i]);
            }
            foreach (var i in new List<string> { "6488", "8069", "3529", "5347", "5274", "6446", "5483", "8299", "3293", "4966" })
            {
                tw50_otc10.Add(_api.Contracts.Stocks["OTC"][i]);
            }
            return _api.Snapshots(tw50_otc10);
        }


        public object Kline(int dayDiff)
        {
            Kbars res = _api.Kbars(_api.Contracts.Futures["TXF"]["TXFR1"],
                // DateTime.Now.AddDays(-dayDiff).ToString("yyyy-MM-dd"),
                // DateTime.Now.ToString("yyyy-MM-dd")
                "2023-12-28", "2023-12-29"
                );

            List<DateTimeOffset> generalTimes = new List<DateTimeOffset>();
            foreach (long timestamp in res.ts)
            {
                DateTime dateTime = DateTimeOffset.FromUnixTimeMilliseconds(timestamp / 1000000).UtcDateTime;
                generalTimes.Add(new DateTimeOffset(dateTime, TimeSpan.Zero));
            }
            List<string> tsFormat = new List<string>();
            foreach (var i in generalTimes) tsFormat.Add(i.ToString("HH:mm:ss"));

            List<Dictionary<string, object>> combinedList = new List<Dictionary<string, object>>();
            for (int i = 0; i < tsFormat.ToArray().Length; i++)
            {
                Dictionary<string, object> dict = new Dictionary<string, object>();
                dict.Add(tsFormat[i], res.Close[i]);
                combinedList.Add(dict);
            }
            return res;
        }

        public Margin Margin()
        {
            var res = _api.Margin();
            return res;
        }   


    // public SJList Focus()
    // {
    //     List<string> todayFocus = ["2353","3231","2344","2324","2356","5521","2352","2382","3481"];
    //     List<string> yesterdayFocus = ["3706","2303","2515","3006","2609","2486","1514","2409","6125"];
    //     List<string> daytradeFocus = ["2382","3231","1519","2353","3017","3661","3006","2356","2376"];

    //     List<IContract> totalFocus = new List<IContract>();
    //     // 九檔全列
    //     foreach (var i in todayFocus){
    //         totalFocus.Add(_api.Contracts.Stocks["TSE"][i]);
    //     }
    //     // 希望重複的會忽略，9+9<=18
    //     foreach (var i in yesterdayFocus){
    //         totalFocus.Add(_api.Contracts.Stocks["TSE"][i]);
    //     }
    //     // 反向取最後九檔，但有可能當沖上榜的跟熱門股重複
    //     foreach (var i in daytradeFocus){
    //         totalFocus.Add(_api.Contracts.Stocks["TSE"][i]);
    //     }
    //     var res = _api.Snapshots(totalFocus);
    //     return res;
    // }    
}


SJCls SJ = new SJCls();
SJ.Login();

// Console.WriteLine(SJ.AmountRank(10));
// Console.WriteLine(SJ.BlueChips());
// Console.WriteLine(SJ.ChangePercentRank(10));
// Console.WriteLine(SJ.Kline(1));
// Console.WriteLine(SJ.Margin());
// Console.WriteLine(SJ.OpPremium("TX1","202401","17900","17800"));
// Console.WriteLine(SJ.TbarTseTxfOtc());
// Console.WriteLine(SJ.TicksLastCount(30));

In [119]:
// _api.Settlements()
// _api.AccountBalance()
// _api.Ticks(_api.Contracts.Stocks["TSE"]["2330"], DateTime.Now.ToString("yyyy-MM-dd"))

// _api.ListPositions(_api.FutureAccount)
// _api.Ticks(_api.Contracts.Futures["TXF"]["TXFR1"], DateTime.Now.ToString("yyyy-MM-dd"))
// _api.Ticks(_api.Contracts.Futures["TXF"]["TXFR1"], DateTime.Now.ToString("yyyy-MM-dd"), TicksQueryType.RangeTime, time_start:"22:40:00",time_end:"22:50:00")
// _api.Ticks(_api.Contracts.Futures["TXF"]["TXFR1"], DateTime.Now.ToString("yyyy-MM-dd"), TicksQueryType.LastCount, last_cnt:10)
// _api.Kbars(_api.Contracts.Futures["TXF"]["TXFR1"],DateTime.Now.AddDays(-1).ToString("yyyy-MM-dd"), DateTime.Now.ToString("yyyy-MM-dd") )
// _api.Contracts.Options["TX1"]["TX120240117950C"]
// _api.ListProfitLossSummary(begin_date: "2023-10-01",end_date: "2023-12-25")
// _api.ListPositions(_api.FutureAccount)
// _api.ListPositionDetail()

<null>

In [7]:
// using System;
// using System.Collections.Generic;
// #r "nuget: Microsoft.Data.Analysis, 0.20.0-preview.22356.1"
// using System.Data;
// using System.Linq;
// using System.Text;
// using System.Collections;
// using System.Threading;
// using System.Threading.Tasks;
// using Microsoft.Data.Analysis;

#r "nuget: Shioaji, 1.1.8"
// _api.Logout()

Installed Packages Shioaji, 1.1.8

In [8]:
using System.IO;
using System.Text.Json;
using Sinopac.Shioaji;

Shioaji _api = new Shioaji();
string jsonString = File.ReadAllText("Sinopac.json");
JsonElement root = JsonDocument.Parse(jsonString).RootElement;
_api.Login(root.GetProperty("API_Key").GetString(), root.GetProperty("Secret_Key").GetString());
_api.ca_activate(@"Sinopac.pfx", root.GetProperty("ca_passwd").GetString(), root.GetProperty("person_id").GetString());

Connecting as shioaji@sinopac on 203.66.91.161:80...
caIsActivated: True


In [22]:
// using Microsoft.AspNetCore.Mvc;
using Newtonsoft.Json;
using System;
using System.Collections.Generic;
using System.Net.Http;
using System.Threading.Tasks;

var client = new HttpClient();
var response = await client.GetAsync("https://openapi.twse.com.tw/v1/" + 
"exchangeReport/MI_INDEX20" // 集中市場每日成交量前二十名證券(會含雜魚ETF)

);
var json = await response.Content.ReadAsStringAsync();
List<dynamic> src = JsonConvert.DeserializeObject<List<dynamic>>(json);

Dictionary<string, Dictionary<string, object>> src_new = new Dictionary<string, Dictionary<string, object>>();
foreach (var item in src)
{
    src_new[item["Code"].ToString()] = new Dictionary<string, object>
    {
        { "Name", item["Name"].ToString()},
        { "ClosingPrice", Convert.ToDouble(item["ClosingPrice"])},
        { "Change", Convert.ToString(item["Change"])} // 平盤好像是空值 ☛ " "，故不能用double
    };
}
src_new.Where(a=>a.Key.Length == 4)
// .Take(3) 
.ElementAt(5)

[5521, System.Collections.Generic.Dictionary`2[System.String,System.Object]] Key 5521 Value key type value Name System.String 工信 ClosingPrice System.Double 14.7 Change System.String 1.30

In [21]:
var client = new HttpClient();
var response = await client.GetAsync("https://openapi.twse.com.tw/v1/" + 
"exchangeReport/FMTQIK" //集中市場每日市場成交資訊
);
var json = await response.Content.ReadAsStringAsync();
List<dynamic> src = JsonConvert.DeserializeObject<List<dynamic>>(json);

foreach(var i in src
.Where(x => x["Date"]=="1121229")
){Console.WriteLine(i);}

// Dictionary<string, Dictionary<string, object>> src_new = new Dictionary<string, Dictionary<string, object>>();
// foreach (var item in src)
// {
//     src_new[item["Date"].ToString()] = new Dictionary<string, object>
//     {
//         { "TradeVolume", item["TradeVolume"].ToString()},
//         { "TradeValue", item["TradeValue"].ToString()},
//         { "Transaction", item["Transaction"].ToString()},
//         { "TAIEX", item["TAIEX"].ToString()},
//         { "Change", item["Change"].ToString()},
//     };
// }
// src_new.Last()

{
  "Date": "1121229",
  "TradeVolume": "5813967380",
  "TradeValue": "272169082532",
  "Transaction": "1971190",
  "TAIEX": "17930.81",
  "Change": "20.44"
}


In [10]:
var client = new HttpClient();
var response = await client.GetAsync("https://openapi.twse.com.tw/v1/" + 
"exchangeReport/TWT84U" //上市個股股價升降幅度
);
var json = await response.Content.ReadAsStringAsync();
List<dynamic> src = JsonConvert.DeserializeObject<List<dynamic>>(json);

// foreach(var i in
// src.Where(x => x.Code.Length == 4)
// src.OrderBy(x => x.PreviousDayPrice).Take(3)
// ){Console.WriteLine(i);}

Dictionary<string, Dictionary<string, object>> src_new = new Dictionary<string, Dictionary<string, object>>();
foreach (var item in src)
{
    src_new[item["Code"].ToString()] = new Dictionary<string, object>
    {
        { "Name", item["Name"].ToString()},
        { "TodayOpeningRefPrice", item["TodayOpeningRefPrice"].ToString()},
        { "PreviousDayOpeningRefPrice", item["PreviousDayOpeningRefPrice"].ToString()},
        { "PreviousDayPrice", item["PreviousDayPrice"].ToString()},
        { "LastTradingDay", item["LastTradingDay"].ToString()},
    };
}
// src_new.ElementAt(5)
src_new.Where(x=>x.Key=="2330" || x.Key=="2454")


// src_new.Where(x=>x.Key=="發行量加權股價指數")
src_new.Take(10)

index value 0 [2330, System.Collections.Generic.Dictionary`2[System.String,System.Object]] Key 2330 Value key value Name 台積電 TodayOpeningRefPrice 593 PreviousDayOpeningRefPrice 593 PreviousDayPrice 593 LastTradingDay 1121229 1 [2454, System.Collections.Generic.Dictionary`2[System.String,System.Object]] Key 2454 Value key value Name 聯發科 TodayOpeningRefPrice 1015 PreviousDayOpeningRefPrice 1025 PreviousDayPrice 1015 LastTradingDay 1121229

In [17]:
var client = new HttpClient();
var response = await client.GetAsync("https://openapi.twse.com.tw/v1/" + 
"exchangeReport/MI_INDEX" //每日收盤行情-大盤統計資訊
);
var json = await response.Content.ReadAsStringAsync();
List<dynamic> src = JsonConvert.DeserializeObject<List<dynamic>>(json);

// foreach(var i in src
// .OrderByDescending(x => x.Percentage)
// .Where(x => x.指數 == "發行量加權股價指數")
// ){Console.WriteLine(i);}

Dictionary<string, Dictionary<string, object>> src_new = new Dictionary<string, Dictionary<string, object>>();
foreach (var item in src)
{
    src_new[item["指數"].ToString()] = new Dictionary<string, object>
    {
        { "收盤指數", item["收盤指數"].ToString()},
        // { "漲跌", item["漲跌"].ToString()},
        // { "漲跌點數", item["漲跌點數"].ToString()},
        { "漲跌百分比", item["漲跌百分比"].ToString()},
        // { "特殊處理註記", item["特殊處理註記"].ToString()},
    };
}
src_new.Where( x=> x.Key=="發行量加權股價指數" || x.Key.Contains("上櫃ESG 30指數"))

index value 0 [發行量加權股價指數, System.Collections.Generic.Dictionary`2[System.String,System.Object]] Key 發行量加權股價指數 Value key value 收盤指數 17931 漲跌百分比 0.11 1 [上櫃ESG 30指數, System.Collections.Generic.Dictionary`2[System.String,System.Object]] Key 上櫃ESG 30指數 Value key value 收盤指數 6195 漲跌百分比 -0.63

In [18]:
var client = new HttpClient();
var response = await client.GetAsync("https://openapi.twse.com.tw/v1/" + 
"exchangeReport/STOCK_DAY_ALL" //上市個股日成交資訊
);
var json = await response.Content.ReadAsStringAsync();
List<dynamic> src = JsonConvert.DeserializeObject<List<dynamic>>(json);

// foreach(var i in src
// .Where(x=> x.Code == "3027" || x.Code == "2454")
// ){Console.WriteLine(i);}

Dictionary<string, Dictionary<string, object>> src_new = new Dictionary<string, Dictionary<string, object>>();
foreach (var item in src)
{
    src_new[item["Code"].ToString()] = new Dictionary<string, object>
    {
        { "Name", item["Name"].ToString()},
        { "OpeningPrice", item["OpeningPrice"].ToString()},
        { "HighestPrice", item["HighestPrice"].ToString()},
        { "LowestPrice", item["LowestPrice"].ToString()},
        { "ClosingPrice", item["ClosingPrice"].ToString()},
        { "Change", item["Change"].ToString()},
        { "TradeValue", item["TradeValue"].ToString()},
    };
}
src_new.Where(x=> x.Key == "2330" || x.Key == "2454")

index value 0 [2330, System.Collections.Generic.Dictionary`2[System.String,System.Object]] Key 2330 Value key value Name 台積電 OpeningPrice 589.00 HighestPrice 593.00 LowestPrice 589.00 ClosingPrice 593.00 Change 0.0000 TradeValue 11931171966 1 [2454, System.Collections.Generic.Dictionary`2[System.String,System.Object]] Key 2454 Value key value Name 聯發科 OpeningPrice 1025.00 HighestPrice 1030.00 LowestPrice 1010.00 ClosingPrice 1015.00 Change -10.0000 TradeValue 5276242225

In [ ]:
using Newtonsoft.Json;
using System;
using System.Collections.Generic;
using System.Net.Http;
using System.Threading.Tasks;

var client = new HttpClient();
var response = await client.GetAsync("https://openapi.twse.com.tw/v1/" + 
"company/applylistingForeign" // 外國公司向證交所申請第一上市之公司
);
var json = await response.Content.ReadAsStringAsync();
List<dynamic> src = JsonConvert.DeserializeObject<List<dynamic>>(json);

foreach(var i in src
// .Where(x=> x["出表日期"] == "1100427" )
){Console.WriteLine(i);}

In [ ]:
new List<string> tw50 = [
"2330","2454","2317","2412","2382","2881","2308","6505","2882","2303",
"3711","2886","2891","1303","1301","2002","1216","2884","2207","5880",
"3008","2892","3045","1326","2357","2885","6669","2395","3034","5871",
"2880","2603","2345","3231","4904","2912","2301","3037","1101","2327",
"2890","2379","2408","3661","3443","4938","5876","2887","2883","1590",
];
var tw50_otc10 = new List<IContract>();
foreach (var i in tw50){
    tw50_otc10.Add(_api.Contracts.Stocks["TSE"][i]);
}
foreach (var i in new List<string>{"6488","8069","3529","5347","5274","6446","5483","8299","3293","4966"}){
    tw50_otc10.Add(_api.Contracts.Stocks["OTC"][i]);
}
// tw50_otc10
_api.Snapshots(tw50_otc10)

In [252]:
// _api.Subscribe(contract:_api.Contracts.Stocks["TSE"]["2330"], quoteType:QuoteType.tick);
_api.Subscribe(_api.Contracts.Stocks["TSE"]["2330"], QuoteType.tick);

List<Dictionary<string, object>> combinedList = new List<Dictionary<string, object>>();
private void myQuoteCB_v1(Exchange exchange, dynamic quote)
{
    Dictionary<string, object> dict = new Dictionary<string, object>();
    dict.Add(quote.GetType().Name, quote);
    combinedList.Add(dict);
}

_api.SetQuoteCallback_v1(myQuoteCB_v1);

Error: System.ObjectDisposedException: Cannot access a disposed object.
Object name: 'Cannot Subscribe with an already disposed session'.
   at SolaceSystems.Solclient.Messaging.Native.SessionImpl.Subscribe(ISubscription subscription, Boolean waitForConfirm)
   at CallSite.Target(Closure, CallSite, Shioaji, Object, QuoteType)
   at System.Dynamic.UpdateDelegates.UpdateAndExecuteVoid3[T0,T1,T2](CallSite site, T0 arg0, T1 arg1, T2 arg2)
   at Submission#252.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
var _contract = _api.Contracts.Futures["MXF"]["MXF202401"];    
var _futOptOrder = new FutOptOrder()
{
    // action = (positions[0].direction == "Buy") ? "Sell" : positions[0].direction,
    action = "Buy",
    price = Convert.ToDouble(16800),
    quantity = 1, price_type = "LMT", order_type = "ROD", octype = OCType.DayTrade
};

var _trade = _api.PlaceOrder(_contract, _futOptOrder);
Console.WriteLine(_trade);